In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VideoGameETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep='\t')
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [ ]:
# Total no of rows in this dataset
df.count()

1785997

## Examine the schema

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
#Drop rows with null values
df=df.dropna()

In [ ]:
#Drop duplicates
df=df.distinct()

In [ ]:
#Check row count after dropping duplicates
df.count()

1785886

In [ ]:
#Change Review date column type from string to date
from pyspark.sql.functions import to_date
 
df = df.withColumn('ReviewDate',to_date(df.review_date, 'yyyy-MM-dd'))
df.printSchema()



root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- ReviewDate: date (nullable = true)



In [ ]:
#drop old review_date column
df=df.drop('review_date')

In [ ]:
# rename new review_date column
df=df.withColumnRenamed('ReviewDate','review_date',)
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# Separate dataframe for Vine Customers
df_vine=df.filter(df['vine']=='Y')

In [ ]:
#Separate dataframe for Non Vine Customers
df_nonvine=df.filter(df['vine']=='N')

## Analysis for vine customers

In [ ]:
#Checking summary statistics of starratings , helpful votes and total votes for vine customers 
vine_summary_analysis_df = df_vine.select(["star_rating","helpful_votes","total_votes"]).describe()

print("Summary statistics for VINE CUSTOMERS")
vine_summary_analysis_df.show()

Summary statistics for VINE CUSTOMERS
+-------+------------------+------------------+------------------+
|summary|       star_rating|     helpful_votes|       total_votes|
+-------+------------------+------------------+------------------+
|  count|              4290|              4290|              4290|
|   mean|4.0748251748251745| 2.348717948717949|3.2783216783216784|
| stddev|0.9183222304948362|12.580450532105264| 13.76996576175815|
|    min|                 1|                 0|                 0|
|    max|                 5|               347|               362|
+-------+------------------+------------------+------------------+



In [ ]:
# Checking total votes for each star ratings
from pyspark.sql.functions import mean, min, max, count,sum
df_vine.groupBy('star_rating').sum('total_votes').show()


+-----------+----------------+
|star_rating|sum(total_votes)|
+-----------+----------------+
|          1|             313|
|          3|            2267|
|          5|            5592|
|          4|            5066|
|          2|             826|
+-----------+----------------+



In [ ]:
# Checking helpful votes for each star ratings
df_vine.groupBy('star_rating').sum('helpful_votes').show()

+-----------+------------------+
|star_rating|sum(helpful_votes)|
+-----------+------------------+
|          1|               103|
|          3|              1419|
|          5|              4245|
|          4|              3861|
|          2|               448|
+-----------+------------------+



In [ ]:
# Checking review count where helpful votes greater than 5
df_vine.select().where(df_vine.helpful_votes>5).count()

295

In [ ]:
# Checking review count where total votes greater than 5
df_vine.select().where(df_vine.total_votes>5).count()

480

In [ ]:
#checking average star rating for products where helpful votes greater than 5
df_vine_star_helpful=df.filter(df_vine.helpful_votes>5)
df_vine_star_helpful.select(mean('star_rating')).show()

+-----------------+
| avg(star_rating)|
+-----------------+
|3.319512572585909|
+-----------------+



In [ ]:
#checking average star rating for products where total votes greater than 5
df_vine_star_total=df.filter(df_vine.total_votes>5)
df_vine_star_total.select(mean('star_rating')).show()

+-----------------+
| avg(star_rating)|
+-----------------+
|3.160227058293443|
+-----------------+



In [ ]:
#Checking number of reviews where total votes and helpful votes both greater than 5
df_vine.select().where((df_vine.total_votes>5) & (df_vine.helpful_votes>5) ).count()

295

In [ ]:
#Checking average starrating for products where total_votes and helpful votes both greater than 5
df_vine_star=df.filter((df_vine.total_votes>5)&(df_vine.helpful_votes>5))
df_vine_star.select(mean('star_rating')).show()

+-----------------+
| avg(star_rating)|
+-----------------+
|3.319512572585909|
+-----------------+



In [ ]:
# Checking number of reviews where star_rating is 5
df_vine_5star=df_vine.filter(df_vine.star_rating==5)
df_vine_5star.select('review_id').distinct().count()

1607

In [ ]:
#Checking 20 products where starrating is 1 with helpful votes in descending order
from pyspark.sql.functions import desc

low_vine_helpful_votes = df_vine.orderBy(df_vine.helpful_votes.desc())
low_vine_helpful_votes = low_vine_helpful_votes.filter('star_rating = 1')

print("Worst 20 rated products with the top helpful votes for VINE CUSTOMERS")
low_vine_helpful_votes.show()

Worst 20 rated products with the top helpful votes for VINE CUSTOMERS
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51640716|R3CMWDEYCJM2O9|B00D6PTMHI|     962335646|      Rocksmith 2014|     Video Games|          1|           24|         31|   Y|                N|Installation conf...|The most confusin...| 2013-12-01|
|         US|   51962742|R29VVB0N4AQPI4|B00D6PTMHI

## Analysis for non_vine custimers

In [ ]:
#Checking summary statistics of starratings , helpful votes and total votes for nonvine customers 
nonvine_summary_analysis_df = df_nonvine.select(["star_rating","helpful_votes","total_votes"]).describe()

print("Summary statistics for NON_VINE CUSTOMERS")
nonvine_summary_analysis_df.show()

Summary statistics for NON_VINE CUSTOMERS
+-------+------------------+------------------+------------------+
|summary|       star_rating|     helpful_votes|       total_votes|
+-------+------------------+------------------+------------------+
|  count|           1781596|           1781596|           1781596|
|   mean|  4.05985981109073|2.2591653775603446| 3.758569282822817|
| stddev|1.3566851626010528|19.185004054278206|22.471849948334242|
|    min|                 1|                 0|                 0|
|    max|                 5|             10498|             10780|
+-------+------------------+------------------+------------------+



In [ ]:
# Checking total votes for each star ratings for nonvine customers
from pyspark.sql.functions import mean, min, max, count,sum
df_nonvine.groupBy('star_rating').sum('total_votes').show()

+-----------+----------------+
|star_rating|sum(total_votes)|
+-----------+----------------+
|          1|         2057802|
|          3|          657016|
|          5|         2462207|
|          4|          954389|
|          2|          564838|
+-----------+----------------+



In [ ]:
# Checking helpful votes for each star ratings for nonvine customers
df_nonvine.groupBy('star_rating').sum('helpful_votes').show()

+-----------+------------------+
|star_rating|sum(helpful_votes)|
+-----------+------------------+
|          1|               103|
|          3|              1419|
|          5|              4245|
|          4|              3861|
|          2|               448|
+-----------+------------------+



In [ ]:
# Checking review count where helpful votes greater than 5 for nonvine customers
df_nonvine.select().where(df_nonvine.helpful_votes>5).count()

148322

In [ ]:
# Checking review count where total votes greater than 5 for nonvine customers
df_nonvine.select().where(df_nonvine.total_votes>5).count()

259189

In [ ]:
#checking average star rating for products where helpful votes greater than 5
df_nonvine_star_helpful=df.filter(df_nonvine.helpful_votes>5)
df_nonvine_star_helpful.select(mean('star_rating')).show()

+-----------------+
| avg(star_rating)|
+-----------------+
|3.319512572585909|
+-----------------+



In [ ]:
#checking average star rating for products where total votes greater than 5
df_nonvine_star_total=df_nonvine.filter(df_nonvine.total_votes>5)
df_nonvine_star_total.select(mean('star_rating')).show()

+------------------+
|  avg(star_rating)|
+------------------+
|3.1587837446805227|
+------------------+



In [ ]:
#Checking number of reviews where total votes and helpful votes both greater than 5
df_nonvine.select().where((df_nonvine.total_votes>5) & (df_nonvine.helpful_votes>5) ).count()

148322

In [ ]:
#Checking average starrating for products where total_votes and helpful votes both greater than 5
df_nonvine_star=df_nonvine.filter((df_nonvine.total_votes>5)&(df_nonvine.helpful_votes>5))
df_nonvine_star.select(mean('star_rating')).show()

+------------------+
|  avg(star_rating)|
+------------------+
|3.3179096829870147|
+------------------+



In [ ]:
# Checking number of reviews where star_rating is 5
df_nonvine_5star=df_nonvine.filter(df_nonvine.star_rating==5)
df_nonvine_5star.count()

1025249

In [ ]:
#Checking 20 products where starrating is 1 with helpful votes in descending order
from pyspark.sql.functions import desc

low_nonvine_helpful_votes = df_nonvine.orderBy(df_nonvine.helpful_votes.desc())
low_nonvine_helpful_votes = low_nonvine_helpful_votes.filter('star_rating = 1')

print("Worst 20 rated products with the top helpful votes for NON VINE CUSTOMERS")
low_nonvine_helpful_votes.show()

Worst 20 rated products with the top helpful votes for NON VINE CUSTOMERS
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48557141|R3HI7AMO8MJ9PF|B007FTE2VW|     265303108|SimCity - Limited...|     Video Games|          1|        10498|      10780|   N|                N|    What a lousy toy|Fundamentally, Si...| 2013-03-06|
|         US|   34072304| RK9RKIUMYF757|B000FK

## Analysis on Amazon Vine Program:

1. We can see that the percentage of 5-star reviews in Vine is not very close to non-Vine reviews (37% for vine to 57.54% for nonvine).

2. Both average star rating for vine and non vine customers is near 4%

3. Both or vine and nonvine customers, numbers of reviews with only helpful votes  greater than 5 and with both helpful votes and total votes greater than 5 are same.

4. Apart from 5 products, all other products rated 1 don't have helpful vote greater than 5 in case of vine customers.

5. In case of both vine and non vine customers average starrating is around 3 for products with helpful and total votes greater than 5. 


So based on the above results it can be considered that Amzon Vine Reviews are not biased.